# Extract respiratory signals with FlowNet2.0

Based on "FlowNet 2.0: Evolution of Optical Flow Estimation with Deep Networks" this notebook extracts respiratory signals from videos using the FlowNet2.0 model. The optical flows are extracted from the videos and the respiratory signals are calculated from the motion vectors.

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

# The scenarios (subject, setting) to process
scenarios = dataset.get_scenarios(['101_natural_lighting'])

In [ ]:
import respiration.utils as utils

flows_dir = utils.dir_path('outputs', 'flownet_flows', mkdir=False)
device = utils.get_torch_device()

In [ ]:
from datetime import datetime

manifest = {
    'timestamp_start': datetime.now(),
    'scenarios': scenarios,
    'device': device,
    'flows': [],
    'incomplete_rois': [],
}

## Part 1: Extract optical flows

This part is heavy on computational resources and may take a long time to complete. The optical flows are extracted from the videos and saved to disk. The extracted optical flows are stored in the `outputs/flownet` directory.

In [ ]:
from respiration.extractor.flownet import (
    FlowNet2,
    FlowNet2C,
    FlowNet2CS,
    FlowNet2CSS,
    FlowNet2S,
    FlowNet2SD,
)

models = {
    'FlowNet2': 'FlowNet2_checkpoint.pth',
    'FlowNet2C': 'FlowNet2-C_checkpoint.pth',
    'FlowNet2CS': 'FlowNet2-CS_checkpoint.pth',
    'FlowNet2CSS': 'FlowNet2-CSS_checkpoint.pth',
    'FlowNet2S': 'FlowNet2-S_checkpoint.pth',
    'FlowNet2SD': 'FlowNet2-SD_checkpoint.pth',
}

In [ ]:
def load_model(name):
    path = utils.file_path('data', 'flownet', models[name])
    loaded = torch.load(path)

    match name:
        case 'FlowNet2':
            model = FlowNet2()
        case 'FlowNet2C':
            model = FlowNet2C()
        case 'FlowNet2CS':
            model = FlowNet2CS()
        case 'FlowNet2CSS':
            model = FlowNet2CSS()
        case 'FlowNet2S':
            model = FlowNet2S()
        case 'FlowNet2SD':
            model = FlowNet2SD()
        case _:
            raise ValueError(f'Unknown model: {name}')

    model.load_state_dict(loaded['state_dict'])
    model = model.to(device)
    model.eval()
    return model

In [ ]:
import os
import torch
import numpy as np

from tqdm.auto import tqdm
from respiration.extractor.flownet import resize_and_center_frames

# Number of frames that are processed at once
batch_size = 10

for (subject, setting) in tqdm(scenarios):
    print(f'Processing {subject} - {setting}')

    video_path = dataset.get_video_path(subject, setting)
    param = utils.get_video_params(video_path)

    if param.width < param.height:
        dimensions = param.height
    else:
        dimensions = param.width

    for model_name in models:
        print(f'  Extracting optical flows with {model_name}...')
        model = load_model(model_name)

        # Store the optical flows vectors (N, 2, H, W). N is the number of frames 
        # in the video minus one, because we calculate the optical flow between consecutive frames.
        optical_flows = np.zeros((param.num_frames - 1, 2, dimensions, dimensions), dtype=np.float32)

        # Extract the optical flow from the video in batches
        for start in range(0, param.num_frames, batch_size - 1):
            num_frames = min(batch_size, param.num_frames - start)

            frames, _ = utils.read_video_rgb(video_path, num_frames, start)
            frames = resize_and_center_frames(frames, dimensions)
            frames = frames.to(device)

            # Fold the frames into (T, C, 2, H, W)
            frames = frames.unfold(0, 2, 1).permute(0, 1, 4, 2, 3)

            with torch.no_grad():
                flows = model(frames)

            for idx in range(flows.shape[0]):
                # Add the optical flow to the numpy array
                optical_flows[start + idx] = flows[idx].cpu().numpy()

            # Garbage collect...
            del frames, flows

        # Store the extracted signals
        filename = f'{subject}_{setting}_{model_name}.npy'
        flow_file = os.path.join(flows_dir, filename)
        np.save(flow_file, optical_flows)

        # Garbage collect the optical flows (8.2GB)
        del optical_flows

        manifest['flows'].append({
            'subject': subject,
            'setting': setting,
            'model': model_name,
            'filename': filename,
        })

In [ ]:
manifest['timestamp_finish'] = datetime.now()

output_dir = utils.dir_path('outputs', 'signals', mkdir=True)
manifest_file = utils.join_paths(output_dir, 'flownet_manifest.json')
utils.write_json(manifest_file, manifest)

## Part 2: Export respiratory signals

This part reads the extracted optical flows and calculates the respiratory signals. The respiratory signals are saved to a CSV file in the `outputs/signals` directory.

In [ ]:
# Read the manifest file
manifest = utils.read_json(manifest_file)

In [ ]:
import os
import numpy as np
import respiration.roi as roi
from tqdm.auto import tqdm

extracted_signals = []

for (subject, setting) in tqdm(scenarios):
    for model_name in models:
        filename = f'{subject}_{setting}_{model_name}.npy'
        flow_file = os.path.join(flows_dir, filename)
        assert os.path.exists(flow_file), f'File not found: {flow_file}'

        optical_flows = np.load(flow_file)

        video_path = dataset.get_video_path(subject, setting)
        params = utils.get_video_params(video_path)
        first_frame = dataset.get_first_frame(subject, setting)
        roi_areas = roi.get_roi_areas(first_frame)
        if len(roi_areas) < 3:
            print(f'Warning: only {len(roi_areas)} ROIs found for {subject} - {setting}')
            manifest['incomplete_rois'].append({
                'subject': subject,
                'setting': setting,
                'rois': [name for (_, name) in roi_areas],
            })

        for ((x, y, w, h), name) in roi_areas:
            # Select the motion vectors in the region of interest (N, 2, H, W)
            flow_region = optical_flows[:, :, y:y + h, x:x + w]

            # Horizontal motion (N, H, W)
            u = flow_region[:, 0, :, :]

            # Vertical motion (N, H, W)
            v = flow_region[:, 1, :, :]

            # Calculate the magnitudes of the motion vectors (N, H, W)
            magnitudes = np.sqrt(u ** 2 + v ** 2)

            # Calculate the mean and standard deviation of the magnitudes
            uv_mean_curve = np.mean(magnitudes, axis=(1, 2))
            uv_std_curve = np.std(magnitudes, axis=(1, 2))

            # Calculate the mean and standard deviation of the vertical motion
            v_mean_curve = v.mean(axis=(1, 2))
            v_std_curve = v.std(axis=(1, 2))

            # Store the extracted signals
            extracted_signals.append({
                'subject': subject,
                'setting': setting,
                'model': model_name,
                'roi': name,
                'sampling_rate': params.fps,
                'signal_uv': uv_mean_curve.tolist(),
                'signal_uv_std': uv_std_curve.tolist(),
                'signal_v': v_mean_curve.tolist(),
                'signal_v_std': v_std_curve.tolist(),
            })

        del optical_flows

In [ ]:
import pandas as pd

signals_df = pd.DataFrame(extracted_signals)
predictions_file = os.path.join(output_dir, 'flownet_predictions.csv')
signals_df.to_csv(predictions_file, index=False)

In [ ]:
signals_df.head()